In [8]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

In [9]:
df = pd.read_csv('Prices.csv')

y = df['Price'].values
x1 = df['Speed'].values
x2 = np.log(df['HardDrive'].values)

a)

In [10]:
with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sigma=1000)
    beta1 = pm.Normal('beta1', mu=0, sigma=1000)
    beta2 = pm.Normal('beta2', mu=0, sigma=1000)
    sigma = pm.HalfNormal('sigma', sigma=1000)

    mu = alpha + beta1 * x1 + beta2 * x2

    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)

    trace = pm.sample(2000, tune=1000, return_inferencedata=True, 
                      random_seed=42, 
                      compile_kwargs={'mode': 'FAST_COMPILE'})

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, sigma]


C:\Users\claud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\
Python312\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 165 seconds.


b)

In [11]:
hdi_beta1 = az.hdi(trace, hdi_prob=0.95)['beta1'].values
hdi_beta2 = az.hdi(trace, hdi_prob=0.95)['beta2'].values

print("b) HDI 95%:")
print(f"beta1: [{hdi_beta1[0]:.4f}, {hdi_beta1[1]:.4f}]")
print(f"beta2: [{hdi_beta2[0]:.4f}, {hdi_beta2[1]:.4f}]")

b) HDI 95%:
beta1: [1.8437, 6.5778]
beta2: [275.0784, 430.1933]


c)

In [12]:
print("\nc) Utilitate predictori:")
print(f"beta1 (Speed): Da, HDI nu contine 0" if hdi_beta1[0] > 0 or hdi_beta1[1] < 0 else "beta1: Nu, HDI contine 0")
print(f"beta2 (log HardDrive): Da, HDI nu contine 0" if hdi_beta2[0] > 0 or hdi_beta2[1] < 0 else "beta2: Nu, HDI contine 0")


c) Utilitate predictori:
beta1 (Speed): Da, HDI nu contine 0
beta2 (log HardDrive): Da, HDI nu contine 0


d)

In [13]:
speed_new = 33
hd_new = np.log(540)

posterior = trace.posterior
alpha_samples = posterior['alpha'].values.flatten()
beta1_samples = posterior['beta1'].values.flatten()
beta2_samples = posterior['beta2'].values.flatten()

mu_new = alpha_samples + beta1_samples * speed_new + beta2_samples * hd_new
hdi_mu = az.hdi(mu_new, hdi_prob=0.90)

print(f"\nd) HDI 90% pentru E(Price) la Speed=33, HardDrive=540:")
print(f"[{hdi_mu[0]:.2f}, {hdi_mu[1]:.2f}]")


d) HDI 90% pentru E(Price) la Speed=33, HardDrive=540:
[2253.31, 2382.73]


e)

In [14]:
sigma_samples = posterior['sigma'].values.flatten()
y_pred = np.random.normal(mu_new, sigma_samples)
hdi_pred = az.hdi(y_pred, hdi_prob=0.90)

print(f"\ne) HDI 90% predictie pentru Price la Speed=33, HardDrive=540:")
print(f"[{hdi_pred[0]:.2f}, {hdi_pred[1]:.2f}]")


e) HDI 90% predictie pentru Price la Speed=33, HardDrive=540:
[1506.05, 3169.31]


Bonus

In [18]:
premium = (df['Premium'] == 'Yes').astype(int).values

with pm.Model() as model_bonus:
    alpha = pm.Normal('alpha', mu=0, sigma=1000)
    beta1 = pm.Normal('beta1', mu=0, sigma=1000)
    beta2 = pm.Normal('beta2', mu=0, sigma=1000)
    beta3 = pm.Normal('beta3', mu=0, sigma=1000)
    sigma = pm.HalfNormal('sigma', sigma=1000)
    
    mu = alpha + beta1 * x1 + beta2 * x2 + beta3 * premium
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    
    trace_bonus = pm.sample(2000, tune=1000, return_inferencedata=True, 
                        random_seed=42, 
                        compile_kwargs={'mode': 'FAST_COMPILE'})
hdi_beta3 = az.hdi(trace_bonus, hdi_prob=0.95)['beta3'].values
print(f"\nBonus: HDI 95% pentru beta3 (Premium): [{hdi_beta3[0]:.2f}, {hdi_beta3[1]:.2f}]")
print(f"Premium afecteaza pretul: {'Da' if hdi_beta3[0] > 0 or hdi_beta3[1] < 0 else 'Nu'}")

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta1, beta2, beta3, sigma]


C:\Users\claud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\
Python312\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 183 seconds.



Bonus: HDI 95% pentru beta3 (Premium): [-1947.56, 1947.75]
Premium afecteaza pretul: Nu
